<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=56ed8a2e27aaffd28d77e9e611b01e42e406906006e16bc3ac1ba2155ae2d5b4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 15:19:26
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.42 C
-------------------
Today PnL: -1.42 L (-0.99%)
Current PnL: -18.17 L (-12.22%)
CY Booked + Current PnL: -6.73 L (-4.53%)
-------------------
Total profit:  1.71 L
Total loss:  -19.88 L
-------------------
Total Booked + Current PnL: 20.25 L (16.64%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.06%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 83.69 L (58.97%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196380.0,31710.0,11626.0,0.14,19.26,5.92,26.31,89.0,2.73,1.41,28.78,X40N,NTT,AC
76,TTKPRESTIG,770.00,100.00,46.0,M-SC,3.32,86146.0,-14631.0,14722.0,-1.01,-14.52,17.09,0.09,245.0,-0.99,0.62,13.18,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-43.66,64.0,H-MC,8.56,111867.0,11363.0,17742.0,-0.25,11.31,15.86,28.96,79.0,0.64,0.81,49.11,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.58,21.61,XY25,NTT,REFINERIES
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198999.0,-1139.0,22029.0,-0.90,-0.57,11.07,10.44,4.0,-0.05,1.43,4.96,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1252.47,80.0,M-SC,6.39,183985.0,2842.0,42629.0,10.36,1.57,23.17,25.10,235.0,0.07,1.33,47.45,XY24,NTT,HEALTHCARE
8,AWL,485.00,-63.71,63.0,H-MC,3.96,258184.0,-43567.0,204998.0,5.21,-14.44,79.40,53.50,116.0,-0.21,1.86,14.68,XY24,NTT,FMCG
80,VAIBHAVGBL,521.00,63.64,65.0,H-SC,6.52,141671.0,-41104.0,152169.0,1.56,-22.49,107.41,60.77,125.0,-0.27,1.02,26.89,XR,NTT,JEWELLERY
31,HINDZINC,730.22,22.12,61.0,M-LC,1.66,199534.0,-5542.0,118104.0,1.25,-2.70,59.19,54.89,52.0,-0.05,1.44,20.74,X5K,ATH,METALS
27,HAPPSTMNDS,1488.71,-13.82,52.0,H-SC,10.22,92960.0,-35205.0,145231.0,1.11,-27.47,156.23,85.85,132.0,-0.24,0.67,7.56,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236520.0,-9631.0,88837.0,-4.60,-3.91,37.56,32.18,16.0,-0.11,1.70,30.72,X200,ATH,IT
53,RAJESHEXPO,518.0,1788.89,52.0,L-SC,2.45,51542.0,-85635.0,85725.0,-3.97,-62.43,166.32,0.07,267.0,-1.00,0.37,28.19,OX40N,NTT,JEWELLERY
81,VALIANTORG,838.0,-318.34,47.0,H-SC,6.00,121414.0,-42191.0,163508.0,-3.64,-25.79,134.67,74.15,139.0,-0.26,0.87,55.63,XR,NTT,CHEMICALS
66,SONACOMS,1006.0,-40.92,37.0,M-SC,11.76,73570.0,-27589.0,102483.0,-3.62,-27.27,139.30,74.03,202.0,-0.27,0.53,3.74,AR,BTT,AUTO
26,GREENPANEL,537.0,223.00,55.0,M-SC,1.27,144253.0,-38825.0,117797.0,-3.27,-21.21,81.66,43.14,230.0,-0.33,1.04,35.69,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-30.62,49.0,H-SC,16.85,198081.0,-1873.0,99100.0,-2.05,-0.94,50.03,48.63,115.0,-0.02,1.43,10.51,XR,NTT,MISC
29,HAVELLS,2069.16,-0.65,54.0,H-MC,1.90,246386.0,-1345.0,76404.0,-0.72,-0.54,31.01,30.30,92.0,-0.02,1.78,12.68,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,100.00,46.0,M-SC,3.32,86146.0,-14631.0,14722.0,-1.01,-14.52,17.09,0.09,245.0,-0.99,0.62,13.18,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,91.37,48.0,H-SC,9.28,120690.0,-21789.0,41312.0,-2.38,-15.29,34.23,13.70,163.0,-0.53,0.87,42.45,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,62.0,H-SC,0.81,226989.0,-42678.0,79015.0,0.14,-15.83,34.81,13.47,138.0,-0.54,1.64,14.81,XY24,NTT,PAINTS
18,CERA,9475.0,-20.53,44.0,H-SC,2.15,144958.0,-30945.0,72972.0,0.61,-17.59,50.34,23.89,149.0,-0.42,1.04,25.19,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-21.19,46.0,H-MC,7.27,106514.0,-25321.0,67541.0,-2.04,-19.21,63.41,32.03,98.0,-0.37,0.77,19.30,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1252.47,80.0,M-SC,6.39,183985.0,2842.0,42629.0,10.36,1.57,23.17,25.10,235.0,0.07,1.33,47.45,XY24,NTT,HEALTHCARE
59,SAIL,228.00,47.55,68.0,M-MC,8.17,233159.0,8197.0,159457.0,-0.59,3.64,68.39,74.53,192.0,0.05,1.68,37.07,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.50,196380.0,31710.0,11626.0,0.14,19.26,5.92,26.31,89.0,2.73,1.41,28.78,X40N,NTT,AC
17,CAMS,5211.76,-1.17,59.0,H-SC,3.10,112384.0,10380.0,33547.0,-2.13,10.18,29.85,43.06,122.0,0.31,0.81,29.03,X40N,ATH,MISC
83,VOLTAS,1530.00,-1.78,46.0,H-MC,2.74,206055.0,14313.0,23449.0,-3.25,7.46,11.38,19.69,99.0,0.61,1.48,14.77,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,64.0,H-MC,8.56,111867.0,11363.0,17742.0,-0.25,11.31,15.86,28.96,79.0,0.64,0.81,49.11,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.17,48.0,M-SC,0.82,99495.0,7996.0,70751.0,-0.84,8.74,71.11,86.07,223.0,0.11,0.72,44.48,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.60,185375.0,5567.0,106850.0,0.05,3.10,57.64,62.52,88.0,0.05,1.34,38.24,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,33.0,H-SC,11.35,129892.0,6556.0,127203.0,-0.76,5.32,97.93,108.45,119.0,0.05,0.94,28.86,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155180.0,4235.0,105228.0,-2.18,2.81,67.81,72.51,53.0,0.04,1.12,9.00,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.84,226002.0,-15922.0,79191.0,-1.90,-6.58,35.04,26.15,8.0,-0.20,1.63,6.80,X40,ATH,IT
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288815.0,-57141.0,131988.0,-3.05,-16.52,45.70,21.64,1.0,-0.43,2.08,2.31,X40,ATH,IT
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.73,218363.0,-33405.0,86625.0,-0.01,-13.27,39.67,21.13,27.0,-0.39,1.57,17.46,X40,ATH,PAINTS
19,CIPLA,1795.00,-20.81,45.0,H-LC,5.48,211240.0,6740.0,34664.0,-2.15,3.30,16.41,20.25,10.0,0.19,1.52,12.32,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236520.0,-9631.0,88837.0,-4.60,-3.91,37.56,32.18,16.0,-0.11,1.70,30.72,X200,ATH,IT
41,ITC,452.0,-38.34,45.0,H-LC,2.28,198999.0,-1139.0,22029.0,-0.90,-0.57,11.07,10.44,4.0,-0.05,1.43,4.96,X40,NTT,FMCG
64,SIEMENS,4671.5,0.81,54.0,H-LC,3.59,162050.0,-24045.0,71529.0,-2.55,-12.92,44.14,25.51,15.0,-0.34,1.17,19.42,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.62,50.0,H-LC,4.24,277992.0,12566.0,44229.0,-0.54,4.73,15.91,21.40,11.0,0.28,2.00,12.41,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.58,21.61,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288815.0,-57141.0,131988.0,-3.05,-16.52,45.70,21.64,1.0,-0.43,2.08,2.31,X40,ATH,IT
79,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.76,228450.0,-5694.0,54325.0,0.06,-2.43,23.78,20.77,86.0,-0.10,1.65,4.32,X40N,NTT,BREWERIES
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198999.0,-1139.0,22029.0,-0.90,-0.57,11.07,10.44,4.0,-0.05,1.43,4.96,X40,NTT,FMCG
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,42.0,H-LC,9.84,226002.0,-15922.0,79191.0,-1.90,-6.58,35.04,26.15,8.0,-0.20,1.63,6.80,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,0.81,54.0,H-LC,3.59,162050.0,-24045.0,71529.0,-2.55,-12.92,44.14,25.51,15.0,-0.34,1.17,19.42,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198999.0,-1139.0,22029.0,-0.90,-0.57,11.07,10.44,4.0,-0.05,1.43,4.96,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,45.0,H-LC,5.48,211240.0,6740.0,34664.0,-2.15,3.30,16.41,20.25,10.0,0.19,1.52,12.32,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.73,218363.0,-33405.0,86625.0,-0.01,-13.27,39.67,21.13,27.0,-0.39,1.57,17.46,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219554.0,9188.0,19584.0,-0.54,4.37,8.92,13.68,37.0,0.47,1.58,21.61,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.25,288815.0,-57141.0,131988.0,-3.05,-16.52,45.70,21.64,1.0,-0.43,2.08,2.31,X40,ATH,IT
39,INFY,2275.00,-18.16,49.0,H-LC,8.19,319521.0,6633.0,165065.0,-2.60,2.12,51.66,54.87,3.0,0.04,2.30,8.91,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198999.0,-1139.0,22029.0,-0.90,-0.57,11.07,10.44,4.0,-0.05,1.43,4.96,X40,NTT,FMCG
82,VBL,671.64,-17.78,38.0,H-LC,6.88,296128.0,-19714.0,132369.0,-2.17,-6.24,44.70,35.67,5.0,-0.15,2.13,6.75,X40N,ATH,FMCG
1,ABB,7934.00,-38.50,59.0,H-LC,9.44,257026.0,-4593.0,123809.0,-1.51,-1.76,48.17,45.57,7.0,-0.04,1.85,9.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7066.67,67.0,L-SC,5.16,78608.0,-15202.0,91822.0,0.38,-16.21,116.81,81.67,269.0,-0.17,0.57,52.23,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.27,82560.0,-30989.0,71043.0,-2.98,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,65.0,M-SC,3.91,84732.0,-17478.0,110872.0,-1.39,-17.10,130.85,91.37,208.0,-0.16,0.61,37.55,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-6.36,42.0,H-SC,12.28,87174.0,-13789.0,108322.0,-2.44,-13.66,124.26,93.63,148.0,-0.13,0.63,29.84,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94500.0,-3480.0,25080.0,-0.11,-3.55,26.54,22.05,152.0,-0.14,0.68,37.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1252.47,80.0,M-SC,6.39,183985.0,2842.0,42629.0,10.36,1.57,23.17,25.10,235.0,0.07,1.33,47.45,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,71.0,H-SC,13.41,225292.0,-45469.0,375427.0,-2.32,-16.79,166.64,121.86,133.0,-0.12,1.62,76.24,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,65.0,M-SC,3.91,84732.0,-17478.0,110872.0,-1.39,-17.10,130.85,91.37,208.0,-0.16,0.61,37.55,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,63.64,65.0,H-SC,6.52,141671.0,-41104.0,152169.0,1.56,-22.49,107.41,60.77,125.0,-0.27,1.02,26.89,XR,NTT,JEWELLERY
49,NATIONALUM,244.55,-43.66,64.0,H-MC,8.56,111867.0,11363.0,17742.0,-0.25,11.31,15.86,28.96,79.0,0.64,0.81,49.11,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.59
1,25,44.17
2,50,75.27


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.86
LC    31.12
MC    24.00
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.98
X40N     11.62
XR       11.47
AR        8.57
XY25      8.41
OX40N     7.97
X200      1.70
MH        1.68
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.94
H-LC    24.55
H-MC    21.03
M-SC    13.39
M-LC     5.52
M-MC     2.67
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.18,-14.85,77.47
FMCG,12.40,-4.24,41.53
FINANCE,10.22,-15.50,65.90
MISC,7.20,-15.14,79.16
BANKS,6.19,-16.21,77.35
PAINTS,5.75,-15.63,32.87
ELECTRICAL,5.60,-6.07,43.29
INSURANCE,3.76,-6.14,45.82
AC,3.60,3.35,12.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3091622.0,22
XR,1317975.0,14
AR,1142066.0,9
X40,768913.0,10
X40N,592414.0,9
OX40N,539620.0,10
XY25,378821.0,6
SR,249385.0,2
X5K,118104.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3635840.0,29
M-SC,1386162.0,17
H-MC,1189395.0,15
H-LC,1117248.0,14
M-LC,385199.0,4
M-MC,307112.0,2
L-SC,248590.0,3
L-MC,60761.0,1
L-LC,38262.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220703.0      6
M-SC       XY24       794363.0      7
H-SC       AR         781750.0      5
           XR         767729.0      7
H-MC       XY24       544573.0      4
H-LC       X40        484898.0      5
M-MC       XY24       307112.0      2
H-SC       X40N       295905.0      4
           OX40N      257298.0      4
           SR         249385.0      2
H-LC       X40N       221358.0      3
H-MC       X40        211664.0      4
H-LC       AR         195338.0      2
H-MC       XY25       165874.0      2
M-SC       AR         164978.0      2
L-SC       XR         162865.0      2
M-LC       XY24       161867.0      2
M-SC       OX40N      129056.0      4
M-LC       X5K        118104.0      1
M-SC       XR         114542.0      2
           XY25       110872.0      1
H-MC       XR         106850.0      1
M-LC       XR         105228.0      1
H-LC       X200        88837.0      1
L-SC       OX40N       85725.0      1
H-MC       X40N        75151.0      2
M-SC       X40         72351.0      1
H-MC       OX40N       67541.0      1
H-LC       XY25        63813.0      2
H-SC       MH          63070.0      1
H-LC       XY24        63004.0      1
L-MC       XR          60761.0      1
L-LC       XY25        38262.0      1
H-MC       MH          17742.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
